# Neo4j MCP Server HTTP Connection Setup (AWS AgentCore)

This notebook demonstrates how to create a Databricks HTTP connection to the Neo4j MCP server deployed on AWS AgentCore Gateway. Once configured, you can query Neo4j graph data directly from SQL using the `http_request` function.

## What This Notebook Does

1. Validates that OAuth2 secrets are configured in Databricks
2. Creates an HTTP connection in Unity Catalog with OAuth2 M2M authentication
3. Tests the connection by calling MCP tools (get-schema, read-cypher)
4. Demonstrates how to parse and use the results

## Prerequisites

- **Databricks Runtime**: 15.4 LTS or later, or SQL warehouse 2023.40+
- **Unity Catalog**: Must be enabled on your workspace
- **AWS AgentCore deployed**: The Neo4j MCP server must be deployed via `neo4j-agentcore-mcp-server`
- **Secrets configured**: Run `setup_databricks_secrets.sh` before this notebook

## Authentication

This integration uses **OAuth2 Machine-to-Machine (M2M)** authentication via AWS Cognito. Databricks handles token exchange and refresh automatically - no manual token management is required.

## Security Note

This integration provides **READ-ONLY** access to Neo4j. The `write-cypher` tool is intentionally excluded to prevent accidental data modifications from analytics workflows.

## Configuration

Update these values to match your environment. The secret scope should match what you used when running `setup_databricks_secrets.sh`.

In [ ]:
# Configuration - update these values for your environment
SECRET_SCOPE = "mcp-neo4j-secrets"           # Must match the scope used in setup_databricks_secrets.sh
CONNECTION_NAME = "neo4j_agentcore_mcp"      # Name for the HTTP connection (metastore-level, not catalog-scoped)

# Tool names are prefixed by AgentCore Gateway with the target name
TOOL_GET_SCHEMA = "neo4j-mcp-server-target___get-schema"
TOOL_READ_CYPHER = "neo4j-mcp-server-target___read-cypher"

## Step 1: Validate Secrets

First, verify that the required OAuth2 secrets exist in Databricks. If this step fails, run `setup_databricks_secrets.sh` from your local machine.

In [ ]:
# Validate that OAuth2 secrets are configured
try:
    gateway_host = dbutils.secrets.get(scope=SECRET_SCOPE, key="gateway_host")
    client_id = dbutils.secrets.get(scope=SECRET_SCOPE, key="client_id")
    client_secret = dbutils.secrets.get(scope=SECRET_SCOPE, key="client_secret")
    token_endpoint = dbutils.secrets.get(scope=SECRET_SCOPE, key="token_endpoint")
    oauth_scope = dbutils.secrets.get(scope=SECRET_SCOPE, key="oauth_scope")
    
    print(f"OAuth2 secrets validated successfully!")
    print(f"  Gateway Host: {gateway_host}")
    print(f"  Client ID: {client_id}")
    print(f"  Client Secret: [REDACTED - {len(client_secret)} characters]")
    print(f"  Token Endpoint: {token_endpoint}")
    print(f"  OAuth Scope: {oauth_scope}")
except Exception as e:
    print(f"ERROR: Failed to retrieve secrets from scope '{SECRET_SCOPE}'")
    print(f"Error: {e}")
    print("")
    print("To fix this, run the setup script from your local machine:")
    print(f"  cd databrick_samples && ./setup_databricks_secrets.sh {SECRET_SCOPE}")
    print("")
    print("Make sure the AgentCore MCP server is deployed first:")
    print("  cd neo4j-agentcore-mcp-server && ./deploy.sh && ./deploy.sh credentials")
    raise

## Step 2: Create the HTTP Connection

Create an HTTP connection in Unity Catalog. This connection:
- Points to the AgentCore Gateway endpoint
- Uses OAuth2 M2M authentication with Cognito credentials
- Databricks automatically handles token exchange and refresh
- Can be shared with other users via Unity Catalog permissions

**Implementation Note**: We use `spark.sql()` with Python f-strings instead of `%%sql` magic because the `CREATE CONNECTION` OPTIONS clause requires constant expressions.

**Scope Note**: Connections are **metastore-level objects** in Unity Catalog, not catalog-scoped. They're shared across all catalogs in your workspace.

**MCP Integration Note**: For full Databricks AI/MCP integration features, you may need to manually enable the "Is mcp connection" checkbox in the Catalog Explorer UI after creating the connection. Navigate to: **Catalog > External Data > Connections > neo4j_agentcore_mcp > Edit** and check the MCP option.

**Note**: If the connection already exists, you'll see an error. Use the cleanup cell at the bottom to drop it first, then re-run this cell.

In [ ]:
# Display connection parameters (secrets are not exposed)
print(f"Creating HTTP connection '{CONNECTION_NAME}'...")
print(f"  Gateway Host: {gateway_host}")
print(f"  Base Path: /mcp")
print(f"  Authentication: OAuth2 M2M (Databricks handles token refresh)")
print(f"  Secret Scope: {SECRET_SCOPE}")

In [ ]:
# Create the HTTP connection with OAuth2 M2M authentication
# Databricks automatically handles token exchange and refresh
# Note: Using spark.sql() because OPTIONS requires constant expressions (no variable interpolation in %%sql)
# Note: Connections are metastore-level objects, not catalog-scoped

create_connection_sql = f"""
CREATE CONNECTION IF NOT EXISTS {CONNECTION_NAME} TYPE HTTP
OPTIONS (
  host secret('{SECRET_SCOPE}', 'gateway_host'),
  base_path '/mcp',
  client_id secret('{SECRET_SCOPE}', 'client_id'),
  client_secret secret('{SECRET_SCOPE}', 'client_secret'),
  oauth_scope secret('{SECRET_SCOPE}', 'oauth_scope'),
  token_endpoint secret('{SECRET_SCOPE}', 'token_endpoint')
)
"""

print("Executing SQL:")
print(create_connection_sql)

spark.sql(create_connection_sql)
print(f"\nConnection created: {CONNECTION_NAME}")

In [ ]:
# Verify connection was created
print(f"Connection '{CONNECTION_NAME}' created successfully!")
print("")
print("=" * 60)
print("IMPORTANT: Manual Step Required for MCP Integration")
print("=" * 60)
print("")
print("The connection is created but won't appear in the Playground")
print("'External MCP Servers' dropdown until you enable the MCP flag:")
print("")
print("  1. Go to Catalog Explorer > External Data > Connections")
print(f"  2. Find '{CONNECTION_NAME}' and click Edit")
print("  3. Check the 'Is MCP connection' checkbox")
print("  4. Save the connection")
print("")
print("After that, you can use it in the Playground:")
print("  Add tools > MCP Servers > External MCP Servers > Unity Catalog Connection")
print("")
print("=" * 60)

## Step 3: Test the Connection - List Tools

The MCP protocol uses JSON-RPC. Let's first list the available tools to verify the connection works.

**Note**: Tool names are prefixed by the AgentCore Gateway with the target name:
- `neo4j-mcp-server-target___get-schema`
- `neo4j-mcp-server-target___read-cypher`

In [ ]:
import json

# MCP JSON-RPC request to list tools
list_tools_request = json.dumps({
    "jsonrpc": "2.0",
    "method": "tools/list",
    "id": 1
})

print("Request payload:")
print(list_tools_request)

In [ ]:
# List available MCP tools
list_tools_sql = f"""
SELECT http_request(
  conn => '{CONNECTION_NAME}',
  method => 'POST',
  path => '',
  headers => map('Content-Type', 'application/json'),
  json => '{{"jsonrpc":"2.0","method":"tools/list","id":1}}'
) AS response
"""

result = spark.sql(list_tools_sql)
display(result)

## Step 4: Get Neo4j Schema

Call the `get-schema` tool to retrieve the Neo4j database schema, including node labels, relationship types, and properties.

**Note**: The tool name is prefixed: `neo4j-mcp-server-target___get-schema`

In [ ]:
# Get the Neo4j database schema using the Gateway-prefixed tool name
get_schema_sql = f"""
SELECT http_request(
  conn => '{CONNECTION_NAME}',
  method => 'POST',
  path => '',
  headers => map('Content-Type', 'application/json'),
  json => '{{"jsonrpc":"2.0","method":"tools/call","params":{{"name":"{TOOL_GET_SCHEMA}","arguments":{{}}}},"id":2}}'
) AS response
"""

result = spark.sql(get_schema_sql)
display(result)

## Step 5: Execute a Read Query

Call the `read-cypher` tool to execute a read-only Cypher query against Neo4j.

**Important**: Only read queries are permitted through this connection. The `write-cypher` tool is intentionally not exposed.

**Note**: The tool name is prefixed: `neo4j-mcp-server-target___read-cypher`

In [ ]:
import json

# Example: Count nodes by label
cypher_query = "MATCH (n) RETURN labels(n) AS label, count(*) AS count ORDER BY count DESC LIMIT 10"

# Build the MCP request with Gateway-prefixed tool name
read_cypher_request = json.dumps({
    "jsonrpc": "2.0",
    "method": "tools/call",
    "params": {
        "name": TOOL_READ_CYPHER,
        "arguments": {
            "query": cypher_query
        }
    },
    "id": 3
})

print("Cypher query:")
print(cypher_query)
print("")
print("MCP request:")
print(read_cypher_request)

In [ ]:
# Execute a read-only Cypher query
# Escape single quotes for SQL
escaped_request = read_cypher_request.replace("'", "''")

cypher_sql = f"""
SELECT http_request(
  conn => '{CONNECTION_NAME}',
  method => 'POST',
  path => '',
  headers => map('Content-Type', 'application/json'),
  json => '{escaped_request}'
) AS response
"""

result = spark.sql(cypher_sql)
display(result)

## Step 6: Parse and Display Results

The MCP response is JSON. Let's parse it and display the results in a more readable format.

In [ ]:
from pyspark.sql.functions import col, from_json, get_json_object
from pyspark.sql.types import StringType, StructType, StructField, ArrayType

# Execute the query and get the response
escaped_request = read_cypher_request.replace("'", "''")

result_df = spark.sql(f"""
    SELECT http_request(
      conn => '{CONNECTION_NAME}',
      method => 'POST',
      path => '',
      headers => map('Content-Type', 'application/json'),
      json => '{escaped_request}'
    ) AS response
""")

# Extract the response body
response_row = result_df.first()
if response_row:
    response = response_row["response"]
    print("Raw response:")
    print(json.dumps(json.loads(response["text"]), indent=2) if "text" in response else response)

## Helper Function: Query Neo4j

Here's a reusable function to query Neo4j through the MCP connection.

In [ ]:
import json

def query_neo4j(cypher_query: str, connection_name: str = None, tool_name: str = None) -> dict:
    """
    Execute a read-only Cypher query against Neo4j via the MCP HTTP connection.
    
    Args:
        cypher_query: The Cypher query to execute (read-only)
        connection_name: Name of the HTTP connection (default: uses CONNECTION_NAME from notebook)
        tool_name: The MCP tool name (default: uses TOOL_READ_CYPHER with Gateway prefix)
    
    Returns:
        dict: The parsed JSON response from Neo4j
    """
    if connection_name is None:
        connection_name = CONNECTION_NAME
    if tool_name is None:
        tool_name = TOOL_READ_CYPHER
    
    # Build the MCP request with Gateway-prefixed tool name
    request_payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": {
                "query": cypher_query
            }
        },
        "id": 1
    })
    
    # Escape single quotes for SQL
    escaped_payload = request_payload.replace("'", "''")
    
    # Execute the query
    result_df = spark.sql(f"""
        SELECT http_request(
          conn => '{connection_name}',
          method => 'POST',
          path => '',
          headers => map('Content-Type', 'application/json'),
          json => '{escaped_payload}'
        ) AS response
    """)
    
    # Parse the response
    response_row = result_df.first()
    if response_row and "response" in response_row.asDict():
        response = response_row["response"]
        if "text" in response:
            return json.loads(response["text"])
    return None

print("Helper function 'query_neo4j' is now available.")
print("")
print("Example usage:")
print(f'  result = query_neo4j("MATCH (n) RETURN count(n) AS total")')
print(f"  # Uses connection: {CONNECTION_NAME}")
print(f"  # Uses tool: {TOOL_READ_CYPHER}")

In [ ]:
# Example: Use the helper function
result = query_neo4j("MATCH (n) RETURN count(n) AS total")
print("Query result:")
print(json.dumps(result, indent=2))

## Troubleshooting

### Common Issues

**1. Secret not found**
```
Secret does not exist with scope: mcp-neo4j-secrets and key: gateway_host
```
Solution: Run `./setup_databricks_secrets.sh` from the `databrick_samples` directory.

**2. Connection already exists**
```
Connection 'neo4j_agentcore_mcp' already exists
```
Solution: Use the cleanup cell below to drop the connection, then re-create it.

**3. HTTP request timeout**
```
Connection timed out
```
Solution: Verify the AgentCore MCP server is running. Check with `cd neo4j-agentcore-mcp-server && ./cloud.sh`.

**4. Authentication failed**
```
401 Unauthorized
```
Solution: The Cognito credentials may be invalid. Re-run:
```
cd neo4j-agentcore-mcp-server && ./deploy.sh credentials
cd ../databrick_samples && ./setup_databricks_secrets.sh
```

**5. Tool not found**
```
Unknown tool: get-schema
```
Solution: Use the Gateway-prefixed tool name: `neo4j-mcp-server-target___get-schema`

## Cleanup

Use this cell to drop the HTTP connection if you need to recreate it or clean up resources.

In [ ]:
# Uncomment the lines below to drop the connection
# spark.sql(f"DROP CONNECTION IF EXISTS {CONNECTION_NAME}")
# print(f"Connection '{CONNECTION_NAME}' dropped.")

## Next Steps

Now that you have a working HTTP connection to Neo4j via AWS AgentCore:

1. **Explore the schema**: Use `get-schema` to understand your graph structure
2. **Write analytics queries**: Use `read-cypher` to query graph data for your analytics
3. **Join with Delta tables**: Combine graph query results with your Delta Lake data
4. **Share the connection**: Grant `USE CONNECTION` to other users via Unity Catalog
5. **Deploy the agent**: Run `neo4j-mcp-agent-deploy.ipynb` to deploy the LangGraph agent

For more information:
- [Databricks HTTP Connections](https://docs.databricks.com/aws/en/query-federation/http)
- [Databricks External MCP Servers](https://docs.databricks.com/aws/en/generative-ai/mcp/external-mcp)
- [Neo4j Cypher Query Language](https://neo4j.com/docs/cypher-manual/current/)
- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)